# 映画の推薦システム
* ここでは推薦アルゴリズムの勉強のために、映画データをもとにした推薦システムを実装する。
* なお、書籍[仕事ではじめる機械学習](http://amzn.to/2DKKS2g)のChap7の[GitHubレポジトリ](https://github.com/oreilly-japan/ml-at-work/blob/master/chap07/Movie_recommendation.ipynb)を参考にしている。
* ここでは`fastFM`というライブラリを用いて、`Factorization Machines`というアルゴリズムで推薦を実装する。

## Factorization Machinesとは
* まず、`Matrix Factorization`というものがある。これは、レーティングの評価行列を、ユーザーの行列とアイテムの行列で表現すること。
    * m x nの評価行列R（m人のユーザー、nこのアイテム）がある時に、これをユーザー行列Uとアイテム行列Iに分解する。
    * このとき、Rは疎な行列。
    * U,Iは密な行列
* 上記の`Matrix Factorization`を一般化したのが`Factorization Machines`。

## Factorization Machinesの特徴
* ユーザーとアイテムの情報以外の特徴量も扱うことができる。
* `Matrix Factorization`と同じく疎な行列を扱うことができる。
* 

## fastFMの動作確認

In [ ]:
* fastFMは

In [5]:
from sklearn.model_selection import learning_curve
import numpy as np
from fastFM import als

from sklearn.feature_extraction import DictVectorizer

train = [
    {"user":"1", "item":"5","age":19},
    {"user":"2", "item":"43","age":33},
    {"user":"3", "item":"20","age":55},
    {"user":"4", "item":"10","age":20},
]

In [8]:
v = DictVectorizer()
X = v.fit_transform(train)
X.toarray()

array([[19.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [33.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       [55.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [20.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

* `DictVectorizer`は文字列型をOneHotに変更してくれるようだ。

In [10]:
y = np.array([5.0, 1.0, 2.0, 4.0])
y

array([5., 1., 2., 4.])

In [15]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X,y)
fm.predict(v.transform({"user":"5","item":"10", "age":24}))

array([3.60775939])